# Demo T1 processing notebook

## Configuration options


Please copy the link from OSF to your data:

In [ ]:
OSF_link = "https://osf.io/ezv74/download/";

Please customize the inversion times (TI) for your measurements (in ms):

In [ ]:
TI = [30, 530, 1030, 1530];

Please specify your data type ("Magnitude" or "Complex"):

In [ ]:
dataType = "Magnitude"

## Download data

In [ ]:
cmd = ['curl -L -o rrsg_dataset.zip', ' ', OSF_link];
[STATUS,MESSAGE] = unix(cmd);
unzip('rrsg_dataset.zip', 'data/');

## Run data format check

In [ ]:
% Reset dataCheckPassed variable
dataCheckPassed = [];

try 
    %% Check that the files have the correct names 
    if exist('data/IRData.mat')
         % File exists.
    else
       error("Your OSF data must contain a file called IRData.mat, and it doesn't.")
    end

    if exist('data/IRMask.mat')
         % File exists.
    else
       error("Your OSF data must contain a file called IRMask.mat, and it doesn't.")
    end

    % Load files
    try
        load('data/IRData.mat');
    catch
        error("IRData.mat could not be loaded correctly.")
    end

    try
        load('data/IRMask.mat');
    catch
        error("IRData.mat could not be loaded correctly.")
    end

    % Check the expected variables exist
    if exist("data")~=1
        error("The IRData file should load a variable called `data`, but it didn't.")
    end
    if exist("Mask")~=1
        error("The IRMask file should load a variable called `Mask`, but it didn't.")
    end

    % Check the dimensions of the files
    if size(data, 4) ~= length(TI)
        error("The fourth dimension of your `data` variable should be the same size as the number of TIs, but it isn't.")
    end
    
    if ~((size(data, 1) == size(Mask, 1)) & (size(data, 2) == size(Mask, 2)) & (size(data, 3) == size(Mask, 3)))
        error("The first three dimensions of you `data` and `Mask` variabls should match, but they don't.")
    end

    % If you got here, all tests passed
    dataCheckPassed = true;
catch err
    error(err)
    dataCheckPassed = false;
end

if dataCheckPassed
    disp("All data format check passed, you can continue to the next step of the script.")
else
    disp("At least one data format check did not pass. Please review our submission guidelines, and reupload your data to OSF.")
end

## Startup qMRLab

In [ ]:
%% MATLAB/OCTAVE CODE
% Adds qMRLab to the path of the environment

try
    cd qMRLab
    startup
catch
    error("qMRLab could not be started correctly.")
end

## Prepare variables for fitting

In [ ]:
% Assign data to variables needed for qMRLab module
IRData = data;
Mask = imrotate(Mask,180);

% Format qMRLab inversion_recovery model parameters, and load them into the Model object
Model = inversion_recovery; 

% Set the customizable settings in the model
Model.Prot.IRData.Mat = [TI'];
Model.options.method = dataType;

% Format data structure so that they may be fit by the model
data = struct();
data.IRData= double(IRData);
data.Mask= double(Mask);

## Fit data

In [ ]:
FitResults = FitData(data,Model,0); % The '0' flag is so that no wait bar is shown.

## Plot data

In [ ]:
%% MATLAB/OCTAVE CODE
% Code used to re-orient the images to make pretty figures, and to assign variables with the axis lengths.

T1_map = imrotate(FitResults.T1.*Mask,-90);
xAxis = [0:size(T1_map,2)-1];
yAxis = [0:size(T1_map,1)-1];

% Raw MRI data at different TI values
TI_0030 = imrotate(squeeze(IRData(:,:,:,1).*Mask),-90);
TI_0530 = imrotate(squeeze(IRData(:,:,:,2).*Mask),-90);
TI_1030 = imrotate(squeeze(IRData(:,:,:,3).*Mask),-90);
TI_1530 = imrotate(squeeze(IRData(:,:,:,4).*Mask),-90);

In [ ]:
%get T1_map --from Octave
%get TI_0030 --from Octave
%get TI_0530 --from Octave
%get TI_1030 --from Octave
%get TI_1530 --from Octave
%get xAxis --from Octave
%get yAxis --from Octave

# PYTHON CODE
# Module imports

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML

from plotly import tools
trace1 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_0030,
                   colorscale='Greys',
                   showscale = False,
                   visible=False,
                   name = 'Signal')
trace2 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_0530,
                   colorscale='Greys',
                   showscale = False,
                   visible=False,
                   name = 'Signal')
trace3 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_1030,
                   colorscale='Greys',
                   showscale = False,
                   visible=True,
                   name = 'Signal')
trace4 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_1530,
                   colorscale='Greys',
                   visible=False,
                   showscale = False,
                   name = 'Signal')
trace5 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=T1_map,
                   colorscale='Portland',
                   xaxis='x2',
                   yaxis='y2',
                   visible=True,
                   name = 'T1 values (ms)')

data=[trace1, trace2, trace3, trace4, trace5]


updatemenus = list([
    dict(active=2,
         x = 0.12,
         xanchor = 'left',
         y = -0.15,
         yanchor = 'bottom',
         direction = 'up',
         font=dict(
                family='Times New Roman',
                size=16
            ),
         buttons=list([   
            dict(label = '30 ms',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, True]},
                         ]),
            dict(label = '530 ms',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, True]},
                         ]),
            dict(label = '1030 ms',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, True]},
                         ]),
            dict(label = '1530 ms',
                 method = 'update',
                 args = [{'visible': [False,False, False, True, True]},
                         ])
        ]),
    )
])

layout = dict(
    width=560,
    height=345,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.06,
            y=1.15,
            showarrow=False,
            text='MR Image',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.6,
            y=1.15,
            showarrow=False,
            text='T<sub>1</sub> map',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.22,
            y=1.15,
            showarrow=False,
            text='T<sub>1</sub> (ms)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.02,
            y=-0.15,
            showarrow=False,
            text='TI:',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 0.6]),
    yaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    xaxis2 = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.4, 1]),
    yaxis2 = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1], anchor='x2'),
    showlegend = False,
    autosize = False,
    updatemenus=updatemenus
)


fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-heatmap', config = config)